# MPI (Message-Passing Interface)


## Overview

* Mechanism of data exchange between processes
* Two types of communication:
 * **point-to-point**: between two processes
 * **collective**: between multiple processes
* Typically only one program, branching depending on the process
* Using the mpi4py Python library

An mpi4py tutorial:
* https://mpi4py.readthedocs.io/en/stable/tutorial.html


Install mpi4py:




In [ ]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.8/464.8 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.0-cp310-cp310-linux_x86_64.whl size=4266268 sha256=d6fdc46335949d615bca63b0ee0f6ded2b4020b8ecacbce66990880f74989d24
  Stored in directory: /root/.cache/pip/wheels/96/17/12/83db63ee0ae5c4b040ee87f2e5c813aea4728b55ec6a37317c
Successfully built mpi4py


## A basic example (no data exchange)

Save as `mpi.py` and run with `mpirun -n 3 python mpi.py`.

In [ ]:
%%writefile mpi.py

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank() # index of the current process
print ("hello from process ", rank)

Overwriting mpi.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -n 4 python mpi.py

hello from process  3
hello from process  2
hello from process  0
hello from process  1


## Point-to-point communication of two processes

### Example: computing $\pi$ with MPI

$$\pi=\sqrt{6\sum_{n=1}^{\infty}\frac{1}{n^2}}$$

**Exercise:** Theoretically estimate the error resulting if we truncate the series at $N$ terms.  

Without MPI:

In [ ]:
import numpy as np
a = np.arange(1,200000)
print (np.sqrt(6*np.sum(1./(a*a))))

3.1415878789259364


### Functions ``send()``, ``recv()``

Save as `mpi.py` and run with `mpirun -n 2 python mpi.py`.

In [ ]:
%%writefile mpi.py

# Evaluate the sum of 2M terms by splitting into two groups of M terms.

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

M = 100
def getPartialSum(start, end):
    a = np.arange(start, end)
    return np.sum(1./(a*a))

s = getPartialSum(1+rank*M, 1+(rank+1)*M)
print ('Process', rank, 'found partial sum from term', 1+rank*M, 'to term', 1+(rank+1)*M-1, ': ', s )

# process 1 sends its partial sum to process 0
if rank == 1:
    comm.send(s, dest=0)

# process 0 receives the partial sum from process 1, adds to its own partial sum
# and outputs the result
elif rank == 0:
    s_other = comm.recv(source=1)
    s_total = s+s_other
    print ('total partial sum =', s_total)
    print ('pi_approx =', np.sqrt(6*s_total))

print ('Process', rank, 'finished')

Overwriting mpi.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -n 2 python mpi.py

Process 0 found partial sum from term 1 to term 100 :  1.6349839001848931
Process 1 found partial sum from term 101 to term 200 :  0.004962645830104402
Process 1 finished
total partial sum = 1.6399465460149976
pi_approx = 3.1368263063309683
Process 0 finished


**Exercise:** Perform the same computation in a "ping-pong" manner: one process sums only even terms, the other only odd terms; after adding a new term the process sends the current result to the other process.  

## Collective communication

Perform efficient (fast, load-balanced) collective operations (e.g., summations) involving multiple processes.

<img src='https://materials.jeremybejarano.com/MPIwithPython/_images/fastSum.png'>

In [ ]:
#from IPython.display import Image
#Image(filename="fastSum.png")

### Function ``gather()``

Pass data from all processes to the chosen process.

Save as `mpi.py` and run with `mpirun -n 5 python mpi.py`.

In [ ]:
%%writefile mpi.py

# Evaluate the sum of MN terms by splitting into M groups of N terms.

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size() # total number of processes
rank = comm.Get_rank()

M = 100
def getPartialSum(start, end):
    a = np.arange(start, end)
    return np.sum(1./(a*a))

s = getPartialSum(1+rank*M, 1+(rank+1)*M)

partialSums = comm.gather(s, root=0)
print ('partialSums gathered at process %d:' %(rank), partialSums)

if rank == 0:
    print ('pi_approx =', np.sqrt(6*np.sum(partialSums)))

Overwriting mpi.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -n 5 python mpi.py

partialSums gathered at process 3: None
partialSums gathered at process 2: None
partialSums gathered at process 1: None
partialSums gathered at process 4: None
partialSums gathered at process 0: [1.6349839001848931, 0.004962645830104402, 0.0016597368826256017, 0.0008309063464401552, 0.0004988762708311448]
pi_approx = 3.1396841231387222


### Function ``bcast()``

Pass data from the chosen process to all other processes.

Save as `mpi.py` and run with `mpirun -n 3 python mpi.py`.

In [ ]:
%%writefile mpi.py

# basic usage of bcast()
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    some_data = {0: 'abcd', 1:1234}
else:
    some_data = None

print ("I'm process", rank, '; data before broadcasting:', some_data)
data = comm.bcast(some_data, root=0)
print ("I'm process", rank, '; data after broadcasting:', data)

Overwriting mpi.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -n 3 python mpi.py

I'm process 0 ; data before broadcasting: {0: 'abcd', 1: 1234}
I'm process 0 ; data after broadcasting: {0: 'abcd', 1: 1234}
I'm process 2 ; data before broadcasting: None
I'm process 1 ; data before broadcasting: None
I'm process 1 ; data after broadcasting: {0: 'abcd', 1: 1234}
I'm process 2 ; data after broadcasting: {0: 'abcd', 1: 1234}


### Functions ``scatter()``, ``reduce()``

* ``scatter()``: distribute data from one source to all processes
* ``reduce()``: combine data from all processes using a collective operation like `sum` or `max`

In [ ]:
%%writefile mpi.py

# Evaluate the sum of N terms by scattering them to N processes.
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data2scatter = [a*a for a in range(1,size+1)]
else:
    data2scatter = None

data = comm.scatter(data2scatter, root=0)

print ('Data at process', rank, ':', data)

b = 1./data

partialSum = comm.reduce(b, op = MPI.SUM, root = 0)

print ('Partial sum at process', rank, ':', partialSum)

if rank == 0:
    result = np.sqrt(6*partialSum)
    print ('Pi_approx:', result)

Overwriting mpi.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -n 5 python mpi.py

Data at process 0 : 1
Data at process 2 : 9
Data at process 3 : 16
Data at process 1 : 4
Data at process 4 : 25
Partial sum at process 3 : None
Partial sum at process 4 : None
Partial sum at process 2 : None
Partial sum at process 1 : None
Partial sum at process 0 : 1.4636111111111112
Pi_approx: 2.9633877010385707


## Example: parallel scalar product
* Generate two random vectors $\mathbf x$ and $\mathbf y$ at the root process. Goal: compute their scalar product $\langle\mathbf x,\mathbf y\rangle$
* Divide $\mathbf x$ and $\mathbf y$ into chunks and scatter them to the other processes
* Compute scalar products between chunks at each process
* Obtain $\langle\mathbf x,\mathbf y\rangle$ by reducing (summing) local scalar products

In [ ]:
%%writefile mpi.py

#"to run" syntax example: mpirun -n 10 python mpi.py 4000000
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#read from command line
N = int(sys.argv[1])    #length of vectors

#arbitrary example vectors, generated to be evenly divided by the number of
#processes for convenience

x = np.random.rand(N) if comm.rank == 0 else None
y = np.random.rand(N) if comm.rank == 0 else None

#initialize as numpy arrays
dot = np.array([0.])
local_N = np.array([0])

#test for conformability
if rank == 0:
                if (N != y.size):
                                print ("vector length mismatch")
                                comm.Abort()

                #currently, our program cannot handle sizes that are not evenly divided by
                #the number of processors
                if (N % size != 0):
                                print ("the number of processors must evenly divide n.")
                                comm.Abort()

                #length of each process's portion of the original vector
                local_N = np.array([N//size])

#communicate local array size to all processes
comm.Bcast(local_N, root=0)

#initialize as numpy arrays
local_x = np.zeros(local_N)
local_y = np.zeros(local_N)

#divide up vectors
comm.Scatter(x, local_x, root=0)
comm.Scatter(y, local_y, root=0)

#local computation of dot product
local_dot = np.array([np.dot(local_x, local_y)])

#sum the results of each
comm.Reduce(local_dot, dot, op = MPI.SUM)

if (rank == 0):
                print ("The dot product computed with MPI:", dot[0])
                print ("The dot product computed w/o  MPI:", np.dot(x,y))

Overwriting mpi.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -n 10 python mpi.py 4000000

The dot product computed with MPI: 1001023.8592246629
The dot product computed w/o  MPI: 1001023.8592246614



**Exercise:** Why is the result $\approx$ 1E6? A bad RNG?

### Reduce-based computation of $\pi$

In [ ]:
%%writefile mpi.py

# run syntax example: mpirun -n 10 python mpi.py 4000000
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#read from command line
N = int(sys.argv[1])    #number of terms

#initialize as numpy array
s = np.array([0.])

#test for conformability
if (N % size != 0):
    print ("the number of processors must evenly divide n.")
    comm.Abort()

#length of each process's portion of the original vector
local_N = np.array([N/size])

def getPartialSum(start, end):
    a = np.arange(start, end)
    return np.sum(1./(a*a))

#local computation of partial sum
local_s = getPartialSum(1+rank*local_N[0], 1+(rank+1)*local_N[0])
local_s = np.array([local_s])

#sum the results of each local sum
comm.Reduce(local_s, s, op = MPI.SUM)

if (rank == 0):
    pi_approx = np.sqrt(6*s[0])
    print ("pi_approx:", pi_approx)

Overwriting mpi.py


The program execution time can be measured with commands `time` or `/usr/bin/time -v`:

(`real`: wall clock time).



In [ ]:
!time mpirun --allow-run-as-root --oversubscribe -n 1 python mpi.py 100000000
!time mpirun --allow-run-as-root --oversubscribe -n 2 python mpi.py 100000000
!time mpirun --allow-run-as-root --oversubscribe -n 4 python mpi.py 100000000

pi_approx: 3.14159264404049

real	0m1.867s
user	0m0.717s
sys	0m1.018s
pi_approx: 3.1415926440404958

real	0m1.603s
user	0m1.454s
sys	0m1.036s
pi_approx: 3.1415926440404975

real	0m2.540s
user	0m2.451s
sys	0m1.856s


Speedup and efficiency of parallelization with 2 processes:

In [ ]:
Speedup = 3.084/1.652
print ('Speedup:', Speedup)

Efficiency = Speedup/2
print ('Efficiency:', Efficiency)

Speedup: 1.8668280871670704
Efficiency: 0.9334140435835352


In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()
print ('cores:', cores)

cores: 2
